In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time 

A partir de l'ID de la playlist, on est capable de récupérer les audio features dans un dataframe. On peut facilement les passer en csv si besoin.

In [4]:
cid = '1344579488b94b5690228b09751fefe5'
secret = 'a049d54e046d48d1a16d73250f733117'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
def getTrackIDs(playlist_id):
    ids = []
    playlist = sp.playlist(playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [6]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)
    return [id, meta['name'], 
             meta['album']['name'],
             meta['album']['artists'][0]['name'],
             meta['album']['release_date'],
             meta['duration_ms'], 
             meta['popularity'], 
             features[0]['acousticness'],
             features[0]['danceability'], 
             features[0]['energy'], 
             features[0]['instrumentalness'],
             features[0]['key'], 
             features[0]['liveness'], 
             features[0]['loudness'], 
             features[0]['mode'], 
             features[0]['speechiness'], 
             features[0]['tempo'], 
             features[0]['time_signature'], 
             features[0]['valence']]

In [7]:
def creat_df_audiofeatures(playlist_id):
    ids = getTrackIDs(playlist_id)
    tracks = []
    for i in range(len(ids)):
#         time.sleep(.5)
        track = getTrackFeatures(ids[i])
        tracks.append(track)
    return pd.DataFrame(tracks, columns = ['id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature','valence']) 

In [8]:
def get_playlist_id(user_id):
    playlists = sp.user_playlists(user_id)
    
    ide=[]
    nom=[]
    while playlists:
        for i, playlist in enumerate(playlists['items']):
            ide.append(playlist['uri'].split(':')[2])
            nom.append(playlist['name'])
        if playlists['next']:
            playlists = sp.next(playlists)
        else:
            playlists = None
    return ide,nom

In [9]:
playlist_id='4pUzBoCxZzig6QncK4fcxD'
playlist_id2 = '6SUm0RDRICmIYYcoccVQta'

In [10]:
source = creat_df_audiofeatures(playlist_id)
playlist_cible = pd.read_csv('data_o.csv')

In [11]:
source.head()

,id,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,7GXG38022Aw5g67p1HTpOp,Hide,Hide,Emilee,2020-07-08,111258,51,0.510,0.417,0.306,0.00046,3,0.127,-13.652,1,0.1980,77.374,4,0.236
1,52APn0QJbjtcHRK0fiXpL6,On brûlera,À peu près,Pomme,2017-10-06,207320,59,0.604,0.487,0.494,0.00000,7,0.204,-7.193,0,0.0266,101.962,3,0.139
2,33IkUVE8cyaWO92IbxIr2j,Old Boy,Fenêtre Sur Rue,Hugo TSR,2012-11-05,213237,42,0.579,0.780,0.650,0.00000,0,0.260,-6.615,1,0.2570,92.979,4,0.560


In [12]:
playlist_cible.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [13]:
i = 0
playlist = [] #prendra l'énergie et l'accousticness de la musique
for i in range (0, len(source)) :
    playlist.append([source["energy"][i], source["artist"][i]])
    playlist_cible.drop(playlist_cible.loc[playlist_cible['id']==source['id'][i]].index, inplace=True)
print(playlist)

[[0.306, 'Emilee'], [0.494, 'Pomme'], [0.65, 'Hugo TSR']]


In [14]:
nouvelle_playlist = pd.DataFrame({})
for i in range(0, len(playlist)):
    nouvelle_playlist = nouvelle_playlist.append(playlist_cible[(playlist_cible["energy"] > playlist[i][0] - 0.1) & (playlist_cible["energy"] < playlist[i][0] + 0.1) & (playlist_cible["acousticness"] > playlist[i][1] - 0.1) & (playlist_cible["acousticness"] < playlist[i][1] + 0.1)])
nouvelle_playlist = nouvelle_playlist.drop_duplicates() #enlever les doublons
nouvelle_playlist.head()

TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [ ]:
#à partir des audiofeatures, pour faire même artiste + un paramètre similaire
donnee = "energy"
i = 0
playlist = [] #prendra l'artiste et la "donnée" de chaque musique
for i in range (0, len(source)) :
    playlist.append([source["artist"][i], source[donnee][i]])
    playlist_cible.drop(playlist_cible.loc[playlist_cible['id']==source.id[i]].index, inplace=True) #enlever de la playlist cible les musiques qu'on a déjà
print(playlist)

nouvelle_playlist = pd.DataFrame({})

for i in range(0, len(playlist)):
    nouvelle_playlist = nouvelle_playlist.append(playlist_cible[(playlist_cible[playlist_cible['artists'].find(playlist[i][1]) =! -1) &  & (playlist_cible[donnee] > playlist[i][1] - 0.1) & (playlist_cible[donnee] < playlist[i][1] + 0.1)])
nouvelle_playlist = nouvelle_playlist.drop_duplicates() #enlever les doublons
nouvelle_playlist.head()

In [15]:
subset(nouvelle_playlist$artist %in% c( "Fats"))


SyntaxError: invalid syntax (<ipython-input-15-0ac48c037e30>, line 1)

In [16]:
playlist_cible.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [17]:
playlist_cible[["artists"]].iloc[[1],:]

,artists
1,['Dennis Day']


In [22]:
print("hello")
print(playlist_cible.iloc[0]['artists'][0])

hello
[


In [32]:
playlist_cible.at[1,'artists']

"['Dennis Day']"

In [15]:
playlist_cible

TypeError: 'method' object is not subscriptable

In [42]:
playlist_cible['artists'].values[0].find(source["artist"].values[1])

-1

In [40]:
source.head()

,id,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,7GXG38022Aw5g67p1HTpOp,Hide,Hide,Emilee,2020-07-08,111258,51,0.510,0.417,0.306,0.00046,3,0.127,-13.652,1,0.1980,77.374,4,0.236
1,52APn0QJbjtcHRK0fiXpL6,On brûlera,À peu près,Pomme,2017-10-06,207320,59,0.604,0.487,0.494,0.00000,7,0.204,-7.193,0,0.0266,101.962,3,0.139
2,33IkUVE8cyaWO92IbxIr2j,Old Boy,Fenêtre Sur Rue,Hugo TSR,2012-11-05,213237,42,0.579,0.780,0.650,0.00000,0,0.260,-6.615,1,0.2570,92.979,4,0.560
